In [142]:
f = open('zauberschule5.txt')
hoehe, breite = [int(x) for x in f.readline().split()]
hoehe, breite

(201, 81)

In [143]:
grid = {(0, x, y): c for x in range(hoehe) for y, c in enumerate(f.readline().strip())}
f.readline()
grid.update({(1, x, y): c for x in range(hoehe) for y, c in enumerate(f.readline().strip())})
G = {v:dict() for v in grid}

In [144]:
def valid(tup):
    z,x,y = tup
    return 0 <= z <= 1 and 0 <= x < hoehe and 0 <= y < breite

In [145]:
dirs = [(1,0,0),(-1,0,0),(0,1,0),(0,-1,0),(0,0,1),(0,0,-1)]
start = None
goal = None
for pos in grid:
    if grid[pos] == 'A':
        start = pos
    if grid[pos] == 'B':
        goal = pos 
    if grid[pos] =='#': continue
    z,x,y = pos
    for zd,xd,yd in dirs:
        zn, xn, yn = z+zd, x+xd, y+yd
        if valid((zn,xn,yn)) and grid[(zn,xn,yn)] != '#':
         
            if z==zn: 
                G[pos][(zn,xn,yn)] = 1
            else:
                G[pos][(zn,xn,yn)] = 3          
     

In [146]:
start, goal

((0, 167, 13), (0, 155, 71))

In [147]:
from heapq import heapify, heappop, heappush

def reconstructPath(s,u,prev):
    temp = []
    while u != s:
        temp.append(u)
        u = prev[u]
    temp.append(s)
    temp.reverse()
    return temp

inf = float('inf')
dist = {v:inf for v in G}
prev = {v:None for v in G}

s = start        # startknoten
dist[s] = 0
endgueltig = set()    

vorlaeufig = [(dist[v],v) for v in G]
heapify(vorlaeufig)

while vorlaeufig:
    _, u = heappop(vorlaeufig)
    if u in endgueltig: continue
    endgueltig.add(u)
    for v in G[u]:
        if dist[v] > dist[u] + G[u][v]:
            dist[v] = dist[u] + G[u][v]
            prev[v] = u
            heappush(vorlaeufig,(dist[v],v))
ziel = goal
print('Pfad von',start,'nach',goal,':',*reconstructPath(start,goal,prev))
print('Distanz:',dist[ziel])

Pfad von (0, 167, 13) nach (0, 155, 71) : (0, 167, 13) (1, 167, 13) (1, 168, 13) (1, 169, 13) (1, 169, 14) (1, 169, 15) (1, 170, 15) (1, 171, 15) (1, 171, 16) (1, 171, 17) (1, 171, 18) (1, 171, 19) (1, 171, 20) (1, 171, 21) (0, 171, 21) (0, 171, 22) (0, 171, 23) (0, 171, 24) (0, 171, 25) (0, 171, 26) (0, 171, 27) (0, 171, 28) (0, 171, 29) (0, 172, 29) (0, 173, 29) (1, 173, 29) (1, 173, 30) (1, 173, 31) (1, 173, 32) (1, 173, 33) (1, 173, 34) (1, 173, 35) (1, 173, 36) (1, 173, 37) (1, 172, 37) (1, 171, 37) (1, 170, 37) (1, 169, 37) (1, 169, 38) (1, 169, 39) (1, 169, 40) (1, 169, 41) (1, 169, 42) (1, 169, 43) (0, 169, 43) (0, 168, 43) (0, 167, 43) (0, 166, 43) (0, 165, 43) (0, 164, 43) (0, 163, 43) (0, 162, 43) (0, 161, 43) (1, 161, 43) (1, 161, 44) (1, 161, 45) (1, 161, 46) (1, 161, 47) (0, 161, 47) (0, 161, 48) (0, 161, 49) (0, 161, 50) (0, 161, 51) (0, 160, 51) (0, 159, 51) (0, 158, 51) (0, 157, 51) (0, 157, 52) (0, 157, 53) (0, 156, 53) (0, 155, 53) (0, 154, 53) (0, 153, 53) (0, 153, 

### A3

Wir lesen ein 3D-Grid ein, das aus zwei Ebenen besteht.

In [11]:
%%writefile input.txt
3 5
#####
# # #
#####

#####
#   #
#####

Overwriting input.txt


In [15]:
f = open('input.txt')
hoehe, breite = [int(x) for x in f.readline().split()]

grid0 = [list(f.readline().strip()) for _ in range(hoehe)]
f.readline()    # leerzeile
grid1 = [list(f.readline().strip()) for _ in range(hoehe)]
f.close()
grid = [grid0,grid1]

In [20]:
def show(grid):
    for row in grid[0]:
        print(*row,sep='')
    print()
    for row in grid[1]:
        print(*row,sep='')

show(grid)

#####
# # #
#####

#####
#   #
#####


Wir können mit Indizes auf die Elemente des 3D-grids zugreifen. Der erste Index ist die Ebene, Index 0 ist die oberste Ebene.

In [22]:
grid[0][1][3] = 'A'   # oberste Ebene, Zeile 1, Spalte 3
show(grid)

#####
# #A#
#####

#####
#   #
#####


Wir können das grid in einen gewichteten Graphen umwandeln, bei dem wir berücksichtigen, dass der Ebenenwechsel mit höherenn Kosten verbunden ist. Die Knoten V des Graphen sind die Koordinaten der begehbaren Punkte.

In [26]:
V = [(z,x,y) for z in range(2) for x in range(hoehe) for y in range(breite) if grid[z][x][y] != '#']
V

[(0, 1, 1), (0, 1, 3), (1, 1, 1), (1, 1, 2), (1, 1, 3)]

Jedem Knoten werden seine Nachbarknoten mit den Kosten zugeordnet.

In [34]:
G = {v: dict() for v in V}
dirs = [(1,0,0),(-1,0,0),(0,1,0),(0,-1,0),(0,0,1),(0,0,-1)]   # mögliche Richtungen
for v in V:
    z,x,y = v
    for zd,xd,yd in dirs:
        zn, xn, yn = z+zd, x+xd, y+yd
        if 0 <= zn < 2 and 0 <= xn < hoehe and 0 <= zn < breite and grid[zn][xn][yn] != '#':
            posn = (zn,xn,yn)
            G[v][posn] = 1 if z == zn else 3

G[(1,1,1)]     

{(0, 1, 1): 3, (1, 1, 2): 1}

G[(1,1,1)] liefert
{(0, 1, 1): 3, (1, 1, 2): 1}, d.h. wir können von (1,1,1) mit Kosten 3 nach (0,1,1) und mit Kosten 1 nach (1,1,2)

Eine Implementation des Dijkstra-Algorithmus findet sich im Abschnitt Infokurs/Graphen1